In [ ]:
# --- STEP 1: IMPORT LIBRARY ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import os

# Nonaktifkan warning ARIMA
warnings.filterwarnings("ignore")

# --- STEP 2: BACA DATA ---
file_path = 'data_pendaftar_jurusan.xlsx'
df = pd.read_excel(file_path)

# --- STEP 3: UBAH FORMAT DATA MENJADI LONG ---
df_long = df.melt(id_vars='Jurusan', var_name='tahun', value_name='jumlah')
df_long['tahun'] = pd.to_numeric(df_long['tahun'], errors='coerce')
df_long['jumlah'] = pd.to_numeric(df_long['jumlah'], errors='coerce')
df_long.dropna(subset=['tahun', 'jumlah'], inplace=True)
df_long['tahun'] = df_long['tahun'].astype(int)

# --- STEP 4: SIAPKAN VARIABEL UNTUK HASIL ---
hasil_arima = []
chart_dir = 'charts_arima'
os.makedirs(chart_dir, exist_ok=True)
jurusan_list = df_long['Jurusan'].unique()

# --- STEP 5: PROSES TIAP JURUSAN ---
for jurusan in jurusan_list:
    data_jurusan = df_long[df_long['Jurusan'] == jurusan].sort_values('tahun')
    tahun = data_jurusan['tahun'].values
    jumlah = data_jurusan['jumlah'].values

    if len(jumlah) < 4:
        continue  # minimal 4 data agar ARIMA bisa jalan

    try:
        # Fit model ARIMA
        model = ARIMA(jumlah, order=(1, 1, 1))
        model_fit = model.fit()

        # Prediksi historis
        pred_hist = model_fit.predict(start=1, end=len(jumlah)-1, typ='levels')
        actual_hist = jumlah[1:]

        # Evaluasi
        mse = mean_squared_error(actual_hist, pred_hist)
        mad = mean_absolute_error(actual_hist, pred_hist)
        mape = np.mean(np.abs((actual_hist - pred_hist) / actual_hist)) * 100

        # Prediksi masa depan (2024, 2025, 2026)
        forecast = model_fit.forecast(steps=3)
        pred_2024, pred_2025, pred_2026 = forecast

        # Simpan hasil evaluasi
        hasil_arima.append({
            'Jurusan': jurusan,
            'Prediksi 2024': round(pred_2024, 2),
            'Prediksi 2025': round(pred_2025, 2),
            'Prediksi 2026': round(pred_2026, 2),
            'MSE': round(mse, 2),
            'MAD': round(mad, 2),
            'MAPE': round(mape, 2)
        })

        # Plot grafik
        plt.figure(figsize=(8, 4))
        plt.plot(tahun, jumlah, marker='o', label='Data Aktual')
        plt.plot(tahun[1:], pred_hist, linestyle='--', label='Prediksi Historis')
        plt.plot([2024, 2025, 2026], forecast, marker='x', color='red', label='Prediksi 2024-2026')
        plt.title(f'Prediksi ARIMA - {jurusan}')
        plt.xlabel('Tahun')
        plt.ylabel('Jumlah Pendaftar')
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        # Simpan grafik
        chart_path = os.path.join(chart_dir, f'{jurusan}.png')
        plt.savefig(chart_path)
        plt.close()

    except Exception as e:
        print(f"Gagal memproses jurusan {jurusan}: {e}")

# --- STEP 6: SIMPAN KE FILE EXCEL ---
output_file = 'hasil_prediksi_ARIMA.xlsx'

try:
    import xlsxwriter
except ImportError:
    import subprocess
    subprocess.check_call(["pip", "install", "xlsxwriter"])
    import xlsxwriter

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Tulis sheet evaluasi
    df_eval = pd.DataFrame(hasil_arima)
    df_eval.to_excel(writer, index=False, sheet_name='Evaluasi ARIMA')

    # Tambah grafik tiap jurusan ke sheet terpisah
    for jurusan in df_eval['Jurusan']:
        sheet_name = jurusan[:31]  # Sheet Excel max 31 karakter
        chart_path = os.path.join(chart_dir, f'{jurusan}.png')
        worksheet = writer.book.add_worksheet(sheet_name)
        writer.sheets[sheet_name] = worksheet
        if os.path.exists(chart_path):
            worksheet.insert_image('B2', chart_path)

print(f"✅ File berhasil disimpan sebagai: {output_file}")


✅ File berhasil disimpan sebagai: hasil_prediksi_ARIMA.xlsx
